![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [679]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


### EDA

In [680]:
ds_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [681]:
numerical_columns = ds_jobs.select_dtypes(include=['int64','float64']).columns

for col in numerical_columns:
    print(f"{col}: {ds_jobs[col].nunique()}")
    print(f"{ds_jobs[col].value_counts()}")
    print()

student_id: 19158
8949     1
10660    1
30726    1
18507    1
31273    1
        ..
11547    1
32067    1
14356    1
18051    1
23834    1
Name: student_id, Length: 19158, dtype: int64

city_development_index: 93
0.920    5200
0.624    2702
0.910    1533
0.926    1336
0.698     683
         ... 
0.649       4
0.807       4
0.781       3
0.625       3
0.664       1
Name: city_development_index, Length: 93, dtype: int64

training_hours: 241
28     329
12     292
18     291
22     282
50     279
      ... 
266      6
234      5
272      5
286      5
238      4
Name: training_hours, Length: 241, dtype: int64

job_change: 2
0.0    14381
1.0     4777
Name: job_change, dtype: int64



In [682]:
categorical_columns = ds_jobs.select_dtypes(include=['object']).columns

for col in categorical_columns:
    print(f"{col}: {ds_jobs[col].nunique()}")
    print(f"{ds_jobs[col].value_counts()}")
    print()

city: 123
city_103    4355
city_21     2702
city_16     1533
city_114    1336
city_160     845
            ... 
city_129       3
city_111       3
city_121       3
city_140       1
city_171       1
Name: city, Length: 123, dtype: int64

gender: 3
Male      13221
Female     1238
Other       191
Name: gender, dtype: int64

relevant_experience: 2
Has relevant experience    13792
No relevant experience      5366
Name: relevant_experience, dtype: int64

enrolled_university: 3
no_enrollment       13817
Full time course     3757
Part time course     1198
Name: enrolled_university, dtype: int64

education_level: 5
Graduate          11598
Masters            4361
High School        2017
Phd                 414
Primary School      308
Name: education_level, dtype: int64

major_discipline: 6
STEM               14492
Humanities           669
Other                381
Business Degree      327
Arts                 253
No Major             223
Name: major_discipline, dtype: int64

experience: 22
>20    

**two-factor categories**: `job_change` and `relevant_experience` has only two options: 0 or 1.

Hence, we can convert these cols to bool data type. 

In [683]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

### Convert numerical columns

In [684]:
# Identify integer cols
int_cols = ds_jobs_transformed.select_dtypes(include=['int64']).columns
print(ds_jobs_transformed[int_cols].dtypes)

student_id        int64
training_hours    int64
dtype: object


In [685]:
# Convert to int32
ds_jobs_transformed[int_cols] = ds_jobs_transformed[int_cols].astype('int32')

# Verify the dtypes
print(ds_jobs_transformed[int_cols].dtypes)

student_id        int32
training_hours    int32
dtype: object


In [686]:
# Identify float cols
float_cols = ds_jobs_transformed.select_dtypes(include=['float64']).columns
print(ds_jobs_transformed[float_cols].dtypes)

city_development_index    float64
job_change                float64
dtype: object


In [687]:
# Convert to float16
ds_jobs_transformed[float_cols] = ds_jobs_transformed[float_cols].astype('float16')

# Verify the dtypes
print(ds_jobs_transformed[float_cols].dtypes)

city_development_index    float16
job_change                float16
dtype: object


### Transform Two-factor Categories columns

In [688]:
print(ds_jobs_transformed['job_change'].unique())

[1. 0.]


In [689]:
print(ds_jobs_transformed['relevant_experience'].unique())

['Has relevant experience' 'No relevant experience']


In [690]:
# Convert relevant_experience col values to 1 or 0
mapping_dict = {
    'Has relevant experience': 1,
    'No relevant experience': 0
}

ds_jobs_transformed['relevant_experience'] = ds_jobs_transformed['relevant_experience'].map(mapping_dict)

print(ds_jobs_transformed['relevant_experience'].unique())

[1 0]


In [691]:
bool_cols = ['job_change', 'relevant_experience']

# Convert type to bool
ds_jobs_transformed[bool_cols] = ds_jobs_transformed[bool_cols].astype('bool')

In [692]:
# Verify the dtypes
ds_jobs_transformed[bool_cols].dtypes

job_change             bool
relevant_experience    bool
dtype: object

### Convert unordered categorical cols

**unordered categories**:
`city`
`gender` 
`major_discipline`
`company_type`

In [693]:
unordered_cat_cols = ['city', 'gender', 'major_discipline', 'company_type']

# Convert type to category
ds_jobs_transformed[unordered_cat_cols] = ds_jobs_transformed[unordered_cat_cols].astype('category')

In [694]:
# Verify the dtypes
ds_jobs_transformed[unordered_cat_cols].dtypes

city                category
gender              category
major_discipline    category
company_type        category
dtype: object

### Convert ordered categories

**ordered categories**:
`education_level`
`experience`
`company_size`
`last_new_job`

In [695]:
# Identify the categories
ds_jobs_transformed['education_level'].value_counts()

Graduate          11598
Masters            4361
High School        2017
Phd                 414
Primary School      308
Name: education_level, dtype: int64

In [696]:
from pandas.api.types import CategoricalDtype

# Define the order of categories
education_order = CategoricalDtype(
    categories=['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'], 
    ordered=True
)
 
# Apply the CategoricalDtype
ds_jobs_transformed['education_level'] = ds_jobs_transformed['education_level'].astype(education_order)

In [697]:
# Verify the dtype
print(ds_jobs_transformed['education_level'].dtypes)
print(ds_jobs_transformed['education_level'].unique())

category
['Graduate', 'Masters', 'High School', NaN, 'Phd', 'Primary School']
Categories (5, object): ['Primary School' < 'High School' < 'Graduate' < 'Masters' < 'Phd']


### Create order list of work exp values

In [698]:
# Analyze experience column
print(ds_jobs_transformed['experience'].unique())
print(ds_jobs_transformed['experience'].info())

# Check missing values
print(ds_jobs_transformed['experience'].isna().sum())

['>20' '15' '5' '<1' '11' '13' '7' '17' '2' '16' '1' '4' '10' '14' '18'
 '19' '12' '3' '6' '9' '8' '20' nan]
<class 'pandas.core.series.Series'>
RangeIndex: 19158 entries, 0 to 19157
Series name: experience
Non-Null Count  Dtype 
--------------  ----- 
19093 non-null  object
dtypes: object(1)
memory usage: 149.8+ KB
None
65


In [699]:
# Fill missing values with 0
ds_jobs_transformed['experience'] = ds_jobs_transformed['experience'].fillna('0')

# Check missing values
print(ds_jobs_transformed['experience'].isna().sum())

0


In [700]:
# Define the categorical order
experience_categories = ['<1'] + [str(i) for i in range(1, 21)] + ['>20']

# Create the CategoricalDtype
experience_dtype = CategoricalDtype(categories=experience_categories, ordered=True)

# Convert 'experience' column to categorical with defined order
ds_jobs_transformed['experience'] = ds_jobs_transformed['experience'].astype(experience_dtype)

In [701]:
ds_jobs_transformed['experience'].unique()

['>20', '15', '5', '<1', '11', ..., '6', '9', '8', '20', NaN]
Length: 23
Categories (22, object): ['<1' < '1' < '2' < '3' ... '18' < '19' < '20' < '>20']

In [702]:
# Analyze company_size column
print(ds_jobs_transformed['company_size'].unique())
print(ds_jobs_transformed['company_size'].info())

# Check missing values
print(ds_jobs_transformed['company_size'].isna().sum())

[nan '50-99' '<10' '10000+' '5000-9999' '1000-4999' '10-49' '100-499'
 '500-999']
<class 'pandas.core.series.Series'>
RangeIndex: 19158 entries, 0 to 19157
Series name: company_size
Non-Null Count  Dtype 
--------------  ----- 
13220 non-null  object
dtypes: object(1)
memory usage: 149.8+ KB
None
5938


In [703]:
# Fill missing values with 0
ds_jobs_transformed['company_size'] = ds_jobs_transformed['company_size'].fillna('<10')

# Check missing values
print(ds_jobs_transformed['company_size'].isna().sum())

0


In [704]:
# Define the categorical order
company_size_categories = ['<10', '10-49', '50-99', '100-499', 
                      '500-999', '1000-4999', '5000-9999', '10000+']

# Create the CategoricalDtype
company_size_dtype = CategoricalDtype(categories=company_size_categories, ordered=True)

# Convert 'company_size' column to categorical with defined order
ds_jobs_transformed['company_size'] = ds_jobs_transformed['company_size'].astype(company_size_dtype)

In [705]:
ds_jobs_transformed['company_size'].unique()

['<10', '50-99', '10000+', '5000-9999', '1000-4999', '10-49', '100-499', '500-999']
Categories (8, object): ['<10' < '10-49' < '50-99' < '100-499' < '500-999' < '1000-4999' < '5000-9999' < '10000+']

In [706]:
# Analyze last_new_job column
print(ds_jobs_transformed['last_new_job'].unique())
print(ds_jobs_transformed['last_new_job'].info())

# Check missing values
print(ds_jobs_transformed['last_new_job'].isna().sum())

['1' '>4' 'never' '4' '3' '2' nan]
<class 'pandas.core.series.Series'>
RangeIndex: 19158 entries, 0 to 19157
Series name: last_new_job
Non-Null Count  Dtype 
--------------  ----- 
18735 non-null  object
dtypes: object(1)
memory usage: 149.8+ KB
None
423


In [707]:
# Fill missing values with 0
ds_jobs_transformed['last_new_job'] = ds_jobs_transformed['last_new_job'].fillna('never')

# Check missing values
print(ds_jobs_transformed['last_new_job'].isna().sum())

0


In [708]:
# Define the categorical order
last_new_job_categories = ['never', '1', '2', '3', '4', '>4']

# Create the CategoricalDtype
last_new_job_dtype = CategoricalDtype(categories=last_new_job_categories, ordered=True)

# Convert 'last_new_job' column to categorical with defined order
ds_jobs_transformed['last_new_job'] = ds_jobs_transformed['last_new_job'].astype(last_new_job_dtype)

In [709]:
# Analyze enrolled_university column
print(ds_jobs_transformed['enrolled_university'].unique())
print(ds_jobs_transformed['enrolled_university'].info())

# Check missing values
print(ds_jobs_transformed['enrolled_university'].isna().sum())

['no_enrollment' 'Full time course' nan 'Part time course']
<class 'pandas.core.series.Series'>
RangeIndex: 19158 entries, 0 to 19157
Series name: enrolled_university
Non-Null Count  Dtype 
--------------  ----- 
18772 non-null  object
dtypes: object(1)
memory usage: 149.8+ KB
None
386


In [710]:
# Fill missing values with 0
ds_jobs_transformed['enrolled_university'] = ds_jobs_transformed['enrolled_university'].fillna('no_enrollment')

# Check missing values
print(ds_jobs_transformed['enrolled_university'].isna().sum())

0


In [711]:
# Define the categorical order
enrolled_university_categories = ['no_enrollment', 'Part time course', 'Full time course']

# Create the CategoricalDtype
enrolled_university_dtype = CategoricalDtype(categories=enrolled_university_categories, ordered=True)

# Convert 'enrolled_university' column to categorical with defined order
ds_jobs_transformed['enrolled_university'] = ds_jobs_transformed['enrolled_university'].astype(enrolled_university_dtype)

In [712]:
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  int32   
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  bool    
 5   enrolled_university     19158 non-null  category
 6   education_level         18698 non-null  category
 7   major_discipline        16345 non-null  category
 8   experience              19093 non-null  category
 9   company_size            19158 non-null  category
 10  company_type            13018 non-null  category
 11  last_new_job            19158 non-null  category
 12  training_hours          19158 non-null  int32   
 13  job_change              19158 non-null  bool    
dtypes: bool(2), category(9

In [713]:
print(ds_jobs.info())
print(ds_jobs_transformed.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

### Filter ordered categorical columns

In [714]:
ds_jobs_transformed = ds_jobs_transformed[(ds_jobs_transformed['experience'] >= '10') \
                                  & (ds_jobs_transformed['company_size'] >= '1000-4999')]

print(ds_jobs_transformed)

       student_id      city  ...  training_hours job_change
9             699  city_103  ...             123      False
12          25619   city_61  ...              23      False
31          22293  city_103  ...             141      False
34          26494   city_16  ...             145      False
40           2547  city_114  ...              14      False
...           ...       ...  ...             ...        ...
19097       25447  city_103  ...              57      False
19101        6803   city_16  ...              89      False
19103       32932   city_10  ...              18      False
19128        3365   city_16  ...              23      False
19143       33047  city_103  ...              18      False

[2201 rows x 14 columns]
